# Performance of Regular Expressions

In [1]:
import re

In [2]:
from time import clock as now

In [12]:
def test(f, *args, **kwargs):
    start = now()
    f(*args, **kwargs)
    print('The function "%s" takes: %fs' % (f.__name__, now() - start))

Testing regex:

In [13]:
def alternation(text):
    pat = re.compile('spa(in|niard)')
    pat.search(text)
test(alternation, "spain")

The function "alternation" takes: 0.000026s


## cProfile:

In [15]:
import cProfile
cProfile.run('alternation("spaniard")')

         7 function calls in 0.000 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.000    0.000    0.000    0.000 <ipython-input-13-ce3077ff9bd0>:1(alternation)
        1    0.000    0.000    0.000    0.000 <string>:1(<module>)
        1    0.000    0.000    0.000    0.000 re.py:221(compile)
        1    0.000    0.000    0.000    0.000 re.py:277(_compile)
        1    0.000    0.000    0.000    0.000 {built-in method exec}
        1    0.000    0.000    0.000    0.000 {method 'disable' of '_lsprof.Profiler' objects}
        1    0.000    0.000    0.000    0.000 {method 'search' of '_sre.SRE_Pattern' objects}




## Debug

In [16]:
re.compile(r'(\w+\d+)+-\d\d', re.DEBUG)

max_repeat 1 4294967295
  subpattern 1
    max_repeat 1 4294967295
      in
        category category_word
    max_repeat 1 4294967295
      in
        category category_digit
literal 45
in
  category category_digit
in
  category category_digit


re.compile(r'(\w+\d+)+-\d\d', re.UNICODE|re.DEBUG)

## Backtracking

In [20]:
def catastrophic(n):
    print('Testing with %d characters' % n)
    pat = re.compile('(a+)+c')
    text = "%s" % ('a'*n)
    pat.search(text)
for n in range(15, 25):
    test(catastrophic, n)

Testing with 15 characters
The function "catastrophic" takes: 0.007167s
Testing with 16 characters
The function "catastrophic" takes: 0.011755s
Testing with 17 characters
The function "catastrophic" takes: 0.011958s
Testing with 18 characters
The function "catastrophic" takes: 0.022251s
Testing with 19 characters
The function "catastrophic" takes: 0.037286s
Testing with 20 characters
The function "catastrophic" takes: 0.090936s
Testing with 21 characters
The function "catastrophic" takes: 0.160596s
Testing with 22 characters
The function "catastrophic" takes: 0.318069s
Testing with 23 characters
The function "catastrophic" takes: 0.595566s
Testing with 24 characters
The function "catastrophic" takes: 1.233160s


In [21]:
def catastrophic(n):
    print('Testing with %d characters' % n)
    pat = re.compile('(x+)+(b+)+c')
    text = 'x'*n
    text += 'b'*n
    pat.search(text)

for n in range(10, 15):
    test(catastrophic, n)

Testing with 10 characters
The function "catastrophic" takes: 0.053931s
Testing with 11 characters
The function "catastrophic" takes: 0.170372s
Testing with 12 characters
The function "catastrophic" takes: 0.640270s
Testing with 13 characters
The function "catastrophic" takes: 2.594830s
Testing with 14 characters
The function "catastrophic" takes: 10.701644s


Regex has a match:

In [22]:
def non_catastrophic(n):
    print('Testing with %d characters' % n)
    pat = re.compile('(x+)+(b+)+c')
    text = 'x'*n
    text += 'b'*n
    text += 'c'
    pat.search(text)

for n in range(12, 18):
    test(non_catastrophic, n)

Testing with 12 characters
The function "non_catastrophic" takes: 0.000048s
Testing with 13 characters
The function "non_catastrophic" takes: 0.000016s
Testing with 14 characters
The function "non_catastrophic" takes: 0.000015s
Testing with 15 characters
The function "non_catastrophic" takes: 0.000014s
Testing with 16 characters
The function "non_catastrophic" takes: 0.000046s
Testing with 17 characters
The function "non_catastrophic" takes: 0.000015s


## Optimisation Recomendations

### Reuse Compiled Patterns

In [34]:
def dontreuse():
    pattern = re.compile(r'\bfoo\b')
    pattern.match('foo bar')


def call100thousandtimes():
    for _ in range(10**5):
        dontreuse()

test(call100thousandtimes)

The function "call100thousandtimes" takes: 0.082300s


In [33]:
pattern = re.compile(r'\bfoo\b')


def reuse():
    pattern.match('foo bar')


def call100thousandtimes():
    for _ in range(10**5):
        reuse()

test(call100thousandtimes)

The function "call100thousandtimes" takes: 0.052349s


### Extract Common Parts in Alternation

In [39]:
pattern = re.compile(r'/(Hello\sWorld|Hello\sContinent|Hello\sCountry)')


def nonoptimized():
    pattern.match('Hello\sCountry')


def call100thousandtimes():
    for _ in range(10**5):
        nonoptimized()

test(call100thousandtimes)

The function "call100thousandtimes" takes: 0.036289s


In [40]:
pattern = re.compile(r'/Hello\s(World|Continent|Country)')


def optimize():
    pattern.match('Hello\sCountry')


def call100thousandtimes():
    for _ in range(10**5):
        optimize()

test(call100thousandtimes)

The function "call100thousandtimes" takes: 0.032092s


### Shortcut to Alternation

In [49]:
pattern = re.compile(r'(white|black|red|blue|green)')


def optimized():
    pattern.match('white')


def call100thousandtimes():
    for _ in range(10**5):
        optimized()

test(call100thousandtimes)

The function "call100thousandtimes" takes: 0.038565s


In [50]:
pattern = re.compile(r'(green|blue|red|black|white)')


def nonoptimized():
    pattern.match('white')


def call100thousandtimes():
    for _ in range(10**5):
        nonoptimized()

test(call100thousandtimes)

The function "call100thousandtimes" takes: 0.041657s


### Use Non-capturing Groups